# Ising-Lenz Ergodicity: Analysis of Time Power-laws in Kappa

    
     (c) 2013, 2014, 2015, 2016, 2025 Süzen
     GPL v3 

$\Omega_{G}(t)$ : Modified TM G-metric, ergodic convergence.  
$\Gamma_{G}(t)$ : Rate of ergodic convergence.  
$\Kappa_{G}(t)$ : Inverse rate of ergodic convergence.   


This notebook plots power-laws and writes out latex table. 
$\Kappa_{G}(t) \sim C t^{\alpha}$

In [ ]:
rm(list=ls()) 
require("isingLenzMC"); # Developed and tested on v0.2.5 and R v4.5.1
library(parallel)       # Core package
library(Hmisc) # for latex tables from data.frame
source("../src/power_utilities.R");

In [ ]:
df_pl_kappa <- readRDS("../datasets/ising1DrateErgodicityPowerlawsTime.rds")
ergoRates <- readRDS("../datasets/ising1DrateErgodicity.rds")
latex(df_pl_kappa, file="../tables/kappa_powerlaw_time.latex") 

In [ ]:
plot_case <- function(result, powerl, meta_on_plot=FALSE) {
    base_plot <- "../plots/powerLawKappa/"
    dynamics <- result$dynamics
    N <- result$N
    ikBT <- result$ikBT
    nrepeat <- powerl$nrepeat
    fsize <- 18
    if(meta_on_plot) fsize <- 14

    jsonlite::write_json(powerl, path = paste(base_plot, dynamics, "N", N, "beta", ikBT, ".json", sep = ""), pretty = TRUE)
    tlog <- seq(powerl$xmin_mean, 7, 0.01) # up-to 6M steps.
    pdf(paste(base_plot, dynamics, "N", N, "beta", ikBT, ".pdf", sep = ""), pointsize = fsize) # pointsize will effect all label and title font sizes.
    #
    # Log-Log plot: First raw Omega' evolution
    #
    lx <- length(result$accept_times_repeats[[1]])
    x <- log10(result$accept_times_repeats[[1]][2:lx])
    y <- log10(1.0 / result$gamma_repeats[[1]][2:lx])
    par(mar=c(5,6,4,1)+.1)
    if(meta_on_plot) {
      layout(matrix(c(1, 2), 2, 1), heights = c(1, 0.1), widths = c(1))
      par(mar = c(12, 10, 6, 4) + 0.1) # clock-wise from the bottom
    }
    plot(10^x, 10^y,
      type = "l", axes = FALSE, ann = FALSE,
      lwd = 0.7, cex.lab = 1.2, cex.axis = 1.2, cex.main = 1, cex.sub = 1,
      log = "xy", lty = 1, xlim = c(10, 1.1e7),
    )
    #
    # Re-write axis labels
    #
    AT <- c(10, 100, 1000, 10000, 100000, 1000000, 10000000)
    AT.label <- expression(10^1, 10^2, 10^3, 10^4, 10^5, 10^6, 10^7)
    AT2 <- c(10^0, 10^1, 10^2, 10^3, 10^4, 10^5, 10^6, 10^7)
    AT2.label <- expression(10^0, 10^1, 10^2, 10^3, 10^4, 10^5, 10^6, 10^7)
    axis(1, at = AT, labels = AT.label)
    axis(2, at = AT2, labels = AT2.label)
    box()
    #
    # Title-subtitle and axis labels
    # Variable parameters are printed, including power-law
    #
    cmean <- sprintf("%.3f", powerl$C_mean)
    Cboot95ci <- sprintf(" [%.3f,%.3f]", powerl$C_lower, powerl$C_upper)

    alphamean <- sprintf("%.3f", powerl$alpha_mean)
    alpheboot95ci <- sprintf(" [%.3f,%.3f]", powerl$alpha_lower, powerl$alpha_upper)


    adjR2mean <- sprintf("%.3f", powerl$adjR2_mean)
    adjR2boot95ci <- sprintf(" [%.3f,%.3f]", powerl$adjR2_lower, powerl$adjR2_upper)

    optDsmean <- sprintf("%.3f", powerl$optDs_mean)
    optDsboot95ci <- sprintf(" [%.3f,%.3f]", powerl$optDs_lower, powerl$optDs_upper)

    title_main <- bquote(paste("Inverse rate power law ", Kappa(t), " ~ ", Ct^{alpha}))
    subtitle <- bquote(paste(
      " N=", .(N), " ", beta, "=", .(ikBT), " ", .(dynamics)
    ))
    ylabel <- bquote(atop("Log Scale Inverse Rate", log[10](K[G])))
    xlabel <- bquote(atop(log[10](t), "Log Scale Monte Carlo Time"))
    title(main = title_main, xlab = xlabel, ylab = ylabel)
    mtext(subtitle)
    #
    # Plot all Omega' independent runs.
    # All Raw functional-trajectories.
    #
    for (i in 2:nrepeat) {
      x <- log10(result$accept_times_repeats[[i]][2:lx])
      y <- log10(1.0 / result$gamma_repeats[[i]][2:lx])
      lines(10^x, 10^y, type = "l", lty = 1)
    }
    #
    # Plot Fitted Power law with CI
    #
    x <- tlog
    y <- powerl$alpha_mean * tlog + powerl$C_mean
    lines(10^x, 10^y, type = "l", col = "red", lwd = 3, lty = 2)
    y <- powerl$alpha_upper * tlog + powerl$C_upper
    lines(10^x, 10^y, type = "l", col = "red", lwd = 3, lty = 3)
    y <- powerl$alpha_lower * tlog + powerl$C_lower
    lines(10^x, 10^y, type = "l", col = "red", lwd = 3, lty = 3)
    #
    # Plot KS-stats law with CI
    # The location, that we start fitting.
    #
    x <- powerl$xmin_mean
    abline(v = 10^x, col = "blue", lwd = 3, lty = 4)
    x <- powerl$xmin_upper
    abline(v = 10^x, col = "blue", lwd = 3, lty = 5)
    x <- powerl$xmin_lower
    abline(v = 10^x, col = "blue", lwd = 3, lty = 5)
    #
    # R puts legends based on color/lty respectively
    # in case of black, though there are nrepeat,
    # it only displays once, which is great!
    #
    legend("topleft",
      legend = c("Power-Law fit", "Time of minimum \n KS-statistic", "MC Runs"),
      col = c("red", "blue", "black"),
      lty = c(2, 4, 1),
      lwd = 2,
      bty = "n",
    )
    if(meta_on_plot) {
      par(mar = c(0, 0, 0, 0))
      plot.new()
      metainfo_power_law <- bquote(paste(K[G], "=", C, t^alpha))
      metainfo_fit_a <- bquote(paste(alpha, "=", .(alphamean), .(alpheboot95ci)))
      metainfo_fit_C <- bquote(paste("log(C)=", .(cmean), .(Cboot95ci)))
      metainfo_fitr2 <- bquote(paste("adj-", R^2, "=", .(adjR2mean), .(adjR2boot95ci)))
      metainfo_fitD <- bquote(paste("KS-stat:", "D=", .(optDsmean), .(optDsboot95ci)))
      mtext(metainfo_power_law, line = 6.0, xpd = TRUE)
      mtext(metainfo_fit_a, line = 4.5, xpd = TRUE)
      mtext(metainfo_fit_C, line = 3.0, xpd = TRUE)
      mtext(metainfo_fitr2, line = 1.5, xpd = TRUE)
      mtext(metainfo_fitD, line = 0.0, xpd = TRUE)
    }
    dev.off()
}


In [ ]:
ncase <- length(ergoRates)
for(i in 1:ncase) {
    result <- ergoRates[[i]]
    powerl <- df_pl_kappa[i,]
    plot_case(result, powerl, meta_on_plot=FALSE) 
}